##IMPORT

In [1]:
import pandas as pd

df = pd.read_csv("model_augmented_complete.csv")
df.head()

,User1,User2,Aggregated_Message,TotalMsgs,AggressiveMsgs,Peerness,Avg_Time_Diff_Seconds,Conversation_CBLabel,Intent_to_Harm,Age_User1,Gender_User1,Grade_User1,Age_User2,Gender_User2,Grade_User2,Aggression_Ratio,Aggregated_Message_Length
0,1,2,You can suck my ass you stupid asshole THIS wa...,36,23,0.500000,9.710811e+05,1,0.769444,11,Others,5,15,Male,9,0.638889,2116
1,1,3,BajaLime BeerGynt bidetofevil will need a uh s...,16,9,0.766667,2.302168e+06,1,0.481250,11,Others,5,11,Male,5,0.562500,812
2,1,4,Hey faggot Seriously I add legit info to pag...,17,8,0.366667,2.100630e+06,0,0.447794,11,Others,5,17,Male,11,0.470588,659
3,1,5,vandalism disagree you limeys are just frus...,24,15,0.366667,1.495599e+06,0,0.612500,11,Others,5,17,Male,11,0.625000,1142
4,1,6,Im all for a poll I dont know why the page was...,15,7,0.433333,2.404213e+06,1,0.420833,11,Others,5,16,Male,10,0.466667,684


##Custom Bert Class and Multi Modal Data Collator

In [2]:
!pip install transformers
from transformers import BertPreTrainedModel, BertModel
from torch import nn
import torch

# Ensure additional_features_dim is available.
# We set it to 3 based on your specific features: Avg_Time, Peerness, Aggression_Ratio
# This global variable is used by the DataCollator, but the model's __init__ will now get it from config.
additional_features_dim = 3

class BertForMultiModalSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)

        # --- MLP for Additional Features ---
        # Get additional_features_dim from config, with a fallback if not explicitly set
        self.additional_features_dim = getattr(config, "additional_features_dim", additional_features_dim)

        # Instead of raw concatenation, we project the 3 features into a
        # higher dimensional space (e.g., 32) to learn feature interactions
        # before merging with BERT.
        self.mlp_hidden_size = 32
        self.feature_mlp = nn.Sequential(
            nn.Linear(self.additional_features_dim, self.mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(self.mlp_hidden_size, self.mlp_hidden_size),
            nn.ReLU()
        )

        self.pre_classifier = nn.Linear(config.hidden_size, config.hidden_size)

        # Classifier input size is now BERT hidden size (768) + MLP output size (32)
        self.classifier = nn.Linear(config.hidden_size + self.mlp_hidden_size, config.num_labels)

        dropout_prob = config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        self.dropout = nn.Dropout(dropout_prob)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None, # Added token_type_ids
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        additional_features=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids, # Passed token_type_ids to the BERT model
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_state = bert_output[0]
        pooled_output = hidden_state[:, 0]

        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = nn.ReLU()(pooled_output)
        pooled_output = self.dropout(pooled_output)

        if additional_features is not None:
            if isinstance(additional_features, list):
                additional_features = torch.tensor(additional_features, dtype=pooled_output.dtype, device=pooled_output.device)

            # --- Pass features through MLP before concatenation ---
            processed_features = self.feature_mlp(additional_features)
            combined_features = torch.cat((pooled_output, processed_features), dim=1)
        else:
            # Fallback if no features provided (though your collator handles this)
            dummy_features = torch.zeros((pooled_output.size(0), self.mlp_hidden_size), device=pooled_output.device)
            combined_features = torch.cat((pooled_output, dummy_features), dim=1)

        logits = self.classifier(combined_features)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + bert_output[1:]
            return ((loss,) + output) if loss is not None else output

        from transformers.modeling_outputs import SequenceClassifierOutput
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=bert_output.hidden_states,
            attentions=bert_output.attentions,
        )



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\mdani\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import DataCollatorWithPadding, AutoTokenizer # Added AutoTokenizer import
import torch

# Re-initialize tokenizer, as it might not be in scope
modelName="bert-base-cased" # Changed from distilbert-base-cased
tokenizer = AutoTokenizer.from_pretrained(modelName)

class MultiModalDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # Separate additional_features from the rest of the features
        additional_features_list = [f.pop('additional_features') for f in features]

        # Use the base DataCollatorWithPadding for the text-related features
        batch = super().__call__(features)

        # Explicitly ensure labels are of type torch.long
        if 'labels' in batch:
            batch['labels'] = batch['labels'].long()

        # Convert additional_features_list to a PyTorch tensor
        # Ensure all sub-lists have the same length
        if additional_features_list:
            try:
                batch['additional_features'] = torch.tensor(additional_features_list, dtype=torch.float32)
            except RuntimeError as e:
                print(f"Error converting additional_features to tensor: {e}")
                print(f"First few additional_features_list entries: {additional_features_list[:5]}")
                raise
        else:
            # Handle empty case: additional_features_dim is defined globally, using it here
            batch['additional_features'] = torch.empty(0, additional_features_dim, dtype=torch.float32)

        return batch

# Instantiate the custom data collator
multi_modal_data_collator = MultiModalDataCollator(tokenizer=tokenizer)

print("MultiModalDataCollator defined and instantiated with 'bert-base-cased' tokenizer. Labels will now be cast to torch.long.")

MultiModalDataCollator defined and instantiated with 'bert-base-cased' tokenizer. Labels will now be cast to torch.long.


## Update Additional Features Extraction Logic

### Subtask:
Extract and normalize 'Avg_Time_Diff_Seconds', 'Peerness', and 'Aggression_Ratio'. This involves calculating min/max for each feature from the `df` and applying min-max scaling to them within the `extract_and_encode_features` function. It will also ensure the `additional_features_dim` is correctly set to 3.


txt col, label col, split, scale, baru boleh tokenize n train, memang macamtu order dia


In [4]:
!pip install datasets
import numpy as np
from datasets import Dataset

# 1. Initialize Dataset
raw_dataset = Dataset.from_pandas(df)
txtCol = "Aggregated_Message"
labelCol = "Conversation_CBLabel"
raw_dataset = raw_dataset.rename_column(labelCol, "labels")

# 2. Split FIRST (Before calculating any stats)
# We split into Train and Temp (Test+Val)
temp_splits = raw_dataset.train_test_split(test_size=0.2, seed=22002423)
train_split = temp_splits['train']
temp_dataset = temp_splits['test']

# Split Temp into Validation and Test
temp_splits_2 = temp_dataset.train_test_split(test_size=0.5, seed=22002423)
test_split = temp_splits_2['train']
validation_split = temp_splits_2['test']

# 3. Calculate Normalization Stats ONLY on the Training Split
# This prevents information from the test/val sets leaking into the scaling logic
print("Calculating min/max stats on TRAINING set only...")

train_avg_time = train_split['Avg_Time_Diff_Seconds']
train_peerness = train_split['Peerness']
train_aggression = train_split['Aggression_Ratio']

stats = {
    'time_min': min(x for x in train_avg_time if x is not None),
    'time_max': max(x for x in train_avg_time if x is not None),
    'peer_min': min(x for x in train_peerness if x is not None),
    'peer_max': max(x for x in train_peerness if x is not None),
    'agg_min': min(x for x in train_aggression if x is not None),
    'agg_max': max(x for x in train_aggression if x is not None),
}

# 4. Define Feature Extraction Function using the Training Stats
def extract_and_encode_features(example):
    # Helper for safe scaling
    def min_max_scale(value, min_val, max_val):
        if value is None: return 0.0
        if max_val == min_val: return 0.0
        return (value - min_val) / (max_val - min_val)

    # Scale using the globally calculated 'stats' dictionary
    scaled_avg_time_diff = min_max_scale(example['Avg_Time_Diff_Seconds'], stats['time_min'], stats['time_max'])
    scaled_peerness = min_max_scale(example['Peerness'], stats['peer_min'], stats['peer_max'])
    scaled_aggression_ratio = min_max_scale(example['Aggression_Ratio'], stats['agg_min'], stats['agg_max'])

    features = [scaled_avg_time_diff, scaled_peerness, scaled_aggression_ratio]
    return {'additional_features': [float(f) for f in features]}

# 5. Apply the function to ALL splits independently
# They are all scaled using the training set's min/max
print("Applying normalization to Train, Validation, and Test sets...")
trainDataset = train_split.map(extract_and_encode_features, batched=False)
validationDataset = validation_split.map(extract_and_encode_features, batched=False)
testDataset = test_split.map(extract_and_encode_features, batched=False)

print("Data splitting and leak-proof normalization complete.")
print(f"Stats used: {stats}")
print(f"Sample Train Features: {trainDataset[0]['additional_features']}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Calculating min/max stats on TRAINING set only...
Applying normalization to Train, Validation, and Test sets...


Map: 100%|██████████| 1447/1447 [00:00<00:00, 3251.40 examples/s]

Data splitting and leak-proof normalization complete.
Stats used: {'time_min': 0.0, 'time_max': 36496680.0, 'peer_min': 0.0, 'peer_max': 0.9966666666666666, 'agg_min': 0.0, 'agg_max': 1.0}
Sample Train Features: [0.18848413061133232, 0.0, 0.2]


we need this step because we will drop every other col except label, text, and additional features

hence if we have more than one additional feature, it will be a column of arrays/lists, each row will be an array of your additional features



##tokenize, train


In [5]:
from transformers import AutoTokenizer # Explicitly import AutoTokenizer

modelName="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(modelName)

def tokenize(examples):
	return tokenizer(
		examples[txtCol], truncation=True, padding=True)

tokenized_train = trainDataset.map(tokenize, batched=True) #map automatically passes in examples from trainDataset etc
tokenized_validation = validationDataset.map(tokenize, batched=True)
tokenized_test = testDataset.map(tokenize, batched=True)

# 3. Remove unnecessary columns from tokenized datasets
columns_to_remove = [
    'User1', 'User2', 'Aggregated_Message', 'TotalMsgs', 'AggressiveMsgs', 'Peerness',
    'Avg_Time_Diff_Seconds', 'Age_User1', 'Gender_User1', 'Grade_User1', 'Age_User2',
    'Gender_User2', 'Grade_User2', 'Aggression_Ratio', 'Aggregated_Message_Length',
    'Intent_to_Harm' # ADDED THIS LINE TO REMOVE INTENT_TO_HARM
]

tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_validation = tokenized_validation.remove_columns(columns_to_remove)
tokenized_test = tokenized_test.remove_columns(columns_to_remove)

# Explicitly cast 'labels' column to integer type, handling None values
def cast_labels_to_long(example):
    # Replace None with 0 (or another appropriate default if 0 is not suitable)
    # Also handles NaN if present as float('nan') by converting to 0 before int cast
    label_value = example['labels']
    if label_value is None or (isinstance(label_value, float) and np.isnan(label_value)):
        example['labels'] = 0 # Default to 0 for None/NaN labels
    else:
        example['labels'] = int(label_value)
    return example

tokenized_train = tokenized_train.map(cast_labels_to_long)
tokenized_validation = tokenized_validation.map(cast_labels_to_long)
tokenized_test = tokenized_test.map(cast_labels_to_long)

print("Datasets re-split, re-tokenized, and cleaned with 'additional_features'.")
print(tokenized_train)
print(tokenized_validation)


Map: 100%|██████████| 1447/1447 [00:00<00:00, 10251.64 examples/s]

Datasets re-split, re-tokenized, and cleaned with 'additional_features'.
Dataset({
    features: ['labels', 'additional_features', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 11576
})
Dataset({
    features: ['labels', 'additional_features', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1448
})



Re-initialize the `BertForMultiModalSequenceClassification` model, the `MultiModalDataCollator`, and the `Trainer`. Then, initiate the training process with the updated setup.


In [ ]:
#!pip install tensorboardX

import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, AutoConfig
from transformers import AutoTokenizer # Ensure tokenizer is available

from huggingface_hub import login
login(token="")

# --- Re-initialize necessary components for trainer setup ---
# Make sure tokenizer is initialized as it's used by DataCollator
modelName="bert-base-cased" # Changed to BERT
tokenizer = AutoTokenizer.from_pretrained(modelName)

# Re-instantiate the custom model
config = AutoConfig.from_pretrained(modelName)
num_labels = 2 # 0 for Non-cyberbullying, 1 for Cyberbullying
config.num_labels = num_labels
config.problem_type = "single_label_classification" # Explicitly set problem type

# Ensure additional_features_dim is defined (it was defined in cell d6f1eff4)
# For robustness, we can redefine it here if necessary, assuming raw_dataset[0]['additional_features'] is available.
# If the kernel restarts, `additional_features_dim` might be lost.
# We will explicitly get it here to ensure it's correct after feature extraction changes.
# Based on current state, additional_features_dim = 1.
# This variable is available in the kernel state.

# Use the newly defined BertForMultiModalSequenceClassification
model = BertForMultiModalSequenceClassification.from_pretrained(modelName, config=config)

# Re-instantiate the custom data collator
multi_modal_data_collator = MultiModalDataCollator(tokenizer=tokenizer)

# Define metrics function (copied from f6148ee9 for completeness)
accuracy=evaluate.load("accuracy")
f1=evaluate.load("f1")
def metrics(evaluate):
    logits, labels = evaluate
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

# Re-define training arguments (copied from f6148ee9 for completeness)
lr = 2e-5
epochs = 2
batchSize = 16
weightDecay = 0.01
output = f"./results/{modelName.replace('/', '_')}_document_classifier"
logging = f"./logs/{modelName.replace('/', '_')}_document_classifier_logs"
repo_name = "rngrye/BERT-cyberbullying-classifier-MLPLeakage"

trainingArgs = TrainingArguments(
    output_dir=output,
    learning_rate=lr,
    per_device_train_batch_size=batchSize,
    per_device_eval_batch_size=batchSize,
    num_train_epochs=epochs,
    weight_decay=weightDecay,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=epochs,
    logging_strategy="epoch",
    logging_dir=logging,
    seed=22002423,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=100,
    report_to="tensorboard",
    push_to_hub=True,
    hub_model_id=repo_name,
    fp16=True,                      # Enable mixed precision for GPU speed
    dataloader_num_workers=4,       # Improve DataLoader throughput
    remove_unused_columns=False,    # Keep `additional_features` for custom model
)

# Re-instantiate the Trainer with the updated datasets and data collator
trainer = Trainer(model=model,
                  args=trainingArgs,
                  train_dataset=tokenized_train,
                  eval_dataset=tokenized_validation,
                  tokenizer=tokenizer,
                  data_collator=multi_modal_data_collator,
                  compute_metrics=metrics)

# --- Explicit device handling and optional pilot run to measure speed ---
import torch, time, math

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print("GPU detected:", torch.cuda.get_device_name(0))
    torch.backends.cudnn.benchmark = True

# Move model explicitly to device (Trainer usually manages this, but being explicit helps)
model.to(device)

# Quick pilot run to measure seconds-per-step (uncomment to run a short test)
run_pilot = False  # Set True to run pilot (200 steps)
if run_pilot:
    total_train_examples = len(tokenized_train)
    effective_batch = batchSize
    steps_per_epoch = math.ceil(total_train_examples / effective_batch)
    total_steps = steps_per_epoch * epochs

    print(f"Approx total steps: {total_steps}")
    start = time.time()
    trainer.train(max_steps=200)
    secs_per_step = (time.time() - start) / 200
    print(f"Seconds per step (pilot): {secs_per_step:.3f}")
    print(f"Estimated total hours (approx): {(secs_per_step * total_steps) / 3600:.2f} h")

# --- Initiate training ---
trainer.train()

Some weights of BertForMultiModalSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'feature_mlp.0.bias', 'feature_mlp.0.weight', 'feature_mlp.2.bias', 'feature_mlp.2.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\mdani\AppData\Local\Temp\ipykernel_3412\2000551512.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


Using device: cpu


c:\Users\mdani\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
''''import importlib, pkgutil
print("torch:", importlib.util.find_spec("torch"))
print("transformers:", importlib.util.find_spec("transformers"))
print("accelerate:", importlib.util.find_spec("accelerate"))''''

torch: ModuleSpec(name='torch', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001522C9DF110>, origin='c:\\Users\\mdani\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\torch\\__init__.py', submodule_search_locations=['c:\\Users\\mdani\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\torch'])
transformers: ModuleSpec(name='transformers', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001522C9DF110>, origin='c:\\Users\\mdani\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\transformers\\__init__.py', submodule_search_locations=['c:\\Users\\mdani\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\transformers'])
accelerate: ModuleSpec(name='accelerate', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001522C9DF110>, origin='c:\\Users\\mdani\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\accelerate\\__init__.py', submodule_search_locations=['c:\\U

#Plot Training and Validation Loss

### Subtask:
Extract the training and validation loss values from the `trainer.state.log_history` after training. Create a plot (e.g., using matplotlib) to visualize how these losses change across training epochs, with appropriate labels and a legend.


**Reasoning**:
To visualize the training and validation loss over epochs, I will import `matplotlib.pyplot`, extract the loss values from the `trainer.state.log_history`, and then plot these values, adding labels, a title, and a legend for clarity.



In [ ]:
import matplotlib.pyplot as plt

# Extract log history from the trainer state
log_history = trainer.state.log_history

# Filter entries for training and validation losses
training_losses = []
validation_losses = []
epochs = []

for log in log_history:
    if 'loss' in log and 'epoch' in log and log['epoch'] not in epochs:
        # Ensure we only add each epoch once for training loss (could be multiple steps per epoch)
        # This assumes 'loss' is logged at the end of each epoch or for specific steps
        training_losses.append(log['loss'])
        epochs.append(log['epoch'])
    if 'eval_loss' in log:
        validation_losses.append(log['eval_loss'])

# Adjust epochs list if needed to match the length of validation_losses
# Assuming validation losses are logged at the end of each epoch
# If there are more training loss entries than validation entries, we might need to align them
# For simplicity, assuming len(training_losses) == len(validation_losses) == len(epochs)
# If not, a more robust epoch matching logic would be needed.

# Get unique sorted epochs for plotting
plot_epochs = sorted(list(set([log['epoch'] for log in log_history if 'eval_loss' in log]))) # Use eval epochs for x-axis

# Filter training losses to match eval epochs if there were more training logs
filtered_training_losses = []
filtered_validation_losses = []

# Collect the last training loss of each epoch and validation loss
for epoch_num in plot_epochs:
    current_epoch_train_losses = [log['loss'] for log in log_history if 'loss' in log and log['epoch'] == epoch_num]
    if current_epoch_train_losses:
        filtered_training_losses.append(current_epoch_train_losses[-1]) # Take the last recorded training loss for the epoch

    current_epoch_eval_losses = [log['eval_loss'] for log in log_history if 'eval_loss' in log and log['epoch'] == epoch_num]
    if current_epoch_eval_losses:
        filtered_validation_losses.append(current_epoch_eval_losses[-1]) # Take the last recorded eval loss for the epoch


# Plotting
plt.figure(figsize=(10, 6))
plt.plot(plot_epochs, filtered_training_losses, label='Training Loss', marker='o')
plt.plot(plot_epochs, filtered_validation_losses, label='Validation Loss', marker='x')

plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.xticks(plot_epochs)
plt.show()

## Evaluate Model on Test Set

### Subtask:
Use the trained `trainer` to make predictions on the `tokenized_test` dataset. Compute and display evaluation metrics (e.g., accuracy, f1-score) for the model's performance on the test set.


**Reasoning**:
To evaluate the model, I need to use the `trainer.predict()` method on the `tokenized_test` dataset, extract the predictions and labels, compute the metrics using the predefined `metrics` function, and then print the results.



In [ ]:
predictions_output = trainer.predict(tokenized_test)
predictions = predictions_output.predictions
label_ids = predictions_output.label_ids

# Compute metrics using the already defined metrics function
eval_metrics = metrics([predictions, label_ids])

print("Test Set Evaluation Metrics:")
print(eval_metrics)

## Generate and Plot Confusion Matrix

### Subtask:
Using the predictions and true labels obtained from the test set evaluation, generate a confusion matrix. Plot the confusion matrix using a heatmap (e.g., with seaborn) to visually represent the classification performance, including labels for true and predicted classes.


**Reasoning**:
To generate and plot the confusion matrix, I first need to convert the raw prediction logits into discrete class labels using `np.argmax`. Then, I will import `matplotlib.pyplot` and `seaborn` for plotting, and `confusion_matrix` from `sklearn.metrics` to calculate the matrix.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Convert raw prediction logits to predicted class labels
predicted_labels = np.argmax(predictions, axis=-1)

# Calculate the confusion matrix
cm = confusion_matrix(label_ids, predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


## Task
Evaluate the trained model on the `tokenized_train` dataset to obtain predictions and true labels. Then, generate and plot a confusion matrix using a heatmap (e.g., with seaborn) for the training set to visually represent its classification performance.

## Evaluate Model on Training Set

### Subtask:
Use the trained `trainer` to make predictions on the `tokenized_train` dataset. This will provide the predicted logits and true labels for the training set.


**Reasoning**:
To evaluate the model on the training set, I will use the `trainer.predict()` method, extract the predictions and labels, compute the metrics using the predefined `metrics` function, and then print the results.



In [ ]:
train_predictions_output = trainer.predict(tokenized_train)
train_predictions = train_predictions_output.predictions
train_label_ids = train_predictions_output.label_ids

# Compute metrics using the already defined metrics function
train_eval_metrics = metrics([train_predictions, train_label_ids])

print("Training Set Evaluation Metrics:")
print(train_eval_metrics)

**Reasoning**:
To visualize the classification performance on the training set, I will convert the prediction logits to labels, calculate the confusion matrix, and then plot it using a heatmap for clear visualization.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Convert raw training prediction logits to predicted class labels
train_predicted_labels = np.argmax(train_predictions, axis=-1)

# Calculate the confusion matrix for the training set
train_cm = confusion_matrix(train_label_ids, train_predicted_labels)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Training Set Confusion Matrix')
plt.show()

saving everything MAKE SURE THIS IS RUN

In [ ]:
# --- Ensure config contains multimodal metadata (overrides if needed) ---
from transformers import AutoConfig
config = AutoConfig.from_pretrained(modelName)
config.additional_features_dim = 3
config.mlp_hidden_size = 8
config.feature_names = ["Avg_Time_Diff", "Peerness", "Aggression_Ratio"]
config.scaling_stats = stats  # min/max stats computed earlier on training set
config.train_dataset_name = "DatasetNameHere"
config.label_mapping = {0: "Non-CB", 1: "CB"}
# Re-create model with updated config (keeps pretrained weights)
model = BertForMultiModalSequenceClassification.from_pretrained(modelName, config=config)
print("Model re-initialized with multimodal config fields.")

# --- Save model, tokenizer, and training metadata for reproducible loading and XAI ---
save_dir = "saved_model"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
import os, json
meta = {
    "scaling_stats": stats,
    "feature_names": config.feature_names,
    "train_dataset_name": config.train_dataset_name,
    "label_mapping": config.label_mapping,
    "mlp_hidden_size": config.mlp_hidden_size,
    "additional_features_dim": config.additional_features_dim,
    "trainer_log_history": trainer.state.log_history
}
with open(os.path.join(save_dir, "training_metadata.json"), "w") as f:
    json.dump(meta, f, default=lambda x: x.tolist() if hasattr(x, "tolist") else x)
print(f"Saved model, tokenizer, and training metadata to {save_dir}")

#test new df
# Task
In order to run the AUC thing, what u need to do is,

* load both the original df and new df, then run all the cells kecuali trainer,

* Then, u just copy paste the new cells and change any variables that arent needed

* ALSO COPY PASTE THE CUSTOM BERT AND DATA COLLATOR in this notebook, since there are some changes yang special for testintg

* INTENT TO HARM

also kalau diff no of features then ada some changes ig
just find the chatgpt convo under fyp title cuda error, then u just say u want to do same thing but for 2 feature or for t5 / gpt

In [1]:
import pandas as pd
df_test = pd.read_csv("test_df.csv")
df_test.head()

FileNotFoundError: [Errno 2] No such file or directory: 'test_df.csv'

In [ ]:
import numpy as np

# --- Inspect labels before filtering ---
labels_before_filter = df_test["Conversation_CBLabel"]

print("Unique labels before filtering:", set(labels_before_filter))
print("Min label before filtering:", labels_before_filter.min())
print("Max label before filtering:", labels_before_filter.max())
print("Label types before filtering:", set(type(x) for x in labels_before_filter))

# --- Filter to binary labels only (0 and 1) ---
original_num_rows = len(df_test)

df_test = df_test[df_test["Conversation_CBLabel"].isin([0, 1])].copy()

filtered_num_rows = len(df_test)

print("\nAfter filtering to only binary labels:")
print(f"Removed {original_num_rows - filtered_num_rows} rows with non-binary labels.")
print("Unique labels after filtering:", set(df_test["Conversation_CBLabel"]))
print("Min label after filtering:", df_test["Conversation_CBLabel"].min())
print("Max label after filtering:", df_test["Conversation_CBLabel"].max())
print("Label types after filtering:", set(type(x) for x in df_test["Conversation_CBLabel"]))

# --- Cast labels to int (required for Trainer / metrics) ---
df_test["Conversation_CBLabel"] = df_test["Conversation_CBLabel"].astype(int)


Unique labels before filtering: {0, 1}
Min label before filtering: 0
Max label before filtering: 1
Label types before filtering: {<class 'int'>}

After filtering to only binary labels:
Removed 0 rows with non-binary labels.
Unique labels after filtering: {0, 1}
Min label after filtering: 0
Max label after filtering: 1
Label types after filtering: {<class 'int'>}


In [ ]:
import torch.nn.functional as F
from transformers import AutoConfig # Added AutoConfig import
# --- Step 2: Load the pre-trained model from Hugging Face ---
repo_name = "rngrye/BERT-cyberbullying-classifier-MLPLeakage"

# Load config from the hub and update relevant parameters
config = AutoConfig.from_pretrained(repo_name)
config.num_labels = 2 # Assuming binary classification
config.problem_type = "single_label_classification"
config.additional_features_dim = 3

# Load the model using the custom class (BertForMultiModalSequenceClassification defined previously)
# The 'additional_features_dim' global variable must be set to 2 as per the model's __init__.
model_from_hub = BertForMultiModalSequenceClassification.from_pretrained(repo_name, config=config)

# Re-instantiate the custom data collator
multi_modal_data_collator = MultiModalDataCollator(tokenizer=tokenizer)



In [ ]:
import numpy as np
from datasets import Dataset
import torch
from transformers import AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import torch.nn.functional as F
import pandas as pd # Import pandas to use pd.isna

# --- Step 1: Prepare df_test for evaluation ---
# Initialize Dataset from df_test
raw_df_test_dataset = Dataset.from_pandas(df_test)

# Ensure txtCol and labelCol are defined (from previous execution if notebook not reset)
# If they were not defined, it would be 'Aggregated_Message' and 'Conversation_CBLabel'
txtCol = "Aggregated_Message"
labelCol = "Conversation_CBLabel"
raw_df_test_dataset = raw_df_test_dataset.rename_column(labelCol, "labels")

# Apply feature extraction to df_test
# The 'extract_and_encode_features' function (defined previously) uses the 'stats' dict from the training set,
# ensuring no data leakage from test set during scaling.
testDataset_prepared = raw_df_test_dataset.map(extract_and_encode_features, batched=False)

# Tokenize df_test using the same tokenizer (bert-base-cased) and tokenize function
modelName="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(repo_name)

# REDEFINE TOKENIZE FUNCTION TO HANDLE POTENTIAL NON-STRING OR NONE VALUES
def tokenize(examples):
    # Ensure all text inputs are strings. Convert non-string or NaN values to empty strings.
    processed_texts = []
    for text_item in examples[txtCol]:
        if pd.isna(text_item) or text_item is None:
            processed_texts.append("")
        else:
            processed_texts.append(str(text_item)) # Ensure it's explicitly a string
    return tokenizer(
        processed_texts, truncation=True, padding=True)

tokenized_df_test = testDataset_prepared.map(tokenize, batched=True)

# Remove unnecessary columns from tokenized_df_test
# The list of columns to remove should be comprehensive, similar to the main training dataset.
# We need to remove all original dataframe columns that are not 'labels', 'additional_features',
# or the tokenized inputs ('input_ids', 'token_type_ids', 'attention_mask').
columns_to_remove_for_df_test = [
    'User1', 'User2', 'Aggregated_Message', 'TotalMsgs', 'AggressiveMsgs', 'Peerness',
    'Avg_Time_Diff_Seconds', 'Aggression_Ratio'
     #, 'Intent_to_Harm','Age_User1', 'Gender_User1', 'Grade_User1', 'Age_User2', 'Gender_User2','Grade_User2', 'Aggregated_Message_Length'
]
tokenized_df_test = tokenized_df_test.remove_columns(columns_to_remove_for_df_test)

# Cast labels to long (using cast_labels_to_long function defined previously)
tokenized_df_test = tokenized_df_test.map(cast_labels_to_long)


print("df_test prepared and tokenized for evaluation:")
print(tokenized_df_test)


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

df_test prepared and tokenized for evaluation:
Dataset({
    features: ['labels', 'additional_features', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 240
})


In [ ]:
def fix_labels(example):
    example["labels"] = int(example["labels"])
    return example

tokenized_df_test = tokenized_df_test.map(fix_labels)


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [ ]:
assert all(len(x) == 3 for x in tokenized_df_test["additional_features"])


# --- Step 3: Define comprehensive metrics function ---
def comprehensive_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Convert logits to probabilities for AUC calculation
    probabilities = F.softmax(torch.from_numpy(logits), dim=-1).numpy()
    # For binary classification (0 or 1), roc_auc_score typically uses the probability of the positive class (class 1)
    positive_class_probabilities = probabilities[:, 1]

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted") # Weighted average suitable for imbalanced classes
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")

    # AUC calculation requires at least two unique labels present in the true labels
    if len(np.unique(labels)) < 2:
        auc = float('nan') # Cannot compute AUC with only one class
    else:
        # Check if positive_class_probabilities has enough variance to compute AUC
        # roc_auc_score might raise a ValueError if all predictions are identical
        if np.all(positive_class_probabilities == positive_class_probabilities[0]):
            auc = float('nan')
        else:
            auc = roc_auc_score(labels, positive_class_probabilities)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc": auc
    }

# --- Step 4: Evaluate the model ---
# Assuming batchSize is defined from previous runs (e.g., batchSize = 16)
# If not, explicitly define it here:
batchSize = 16

evaluation_args = TrainingArguments(
    output_dir="./eval_results_df_test", # Dummy output directory for evaluation logs
    do_train=False, # We are only evaluating
    do_eval=True,
    per_device_eval_batch_size=batchSize,
    report_to="none", # Do not report this specific evaluation to any external service
    remove_unused_columns=False, # Important to keep 'additional_features' column for the custom model
)

# Initialize Trainer for evaluation purposes
evaluator = Trainer(
    model=model_from_hub,
    args=evaluation_args,
    tokenizer=tokenizer,
    data_collator=multi_modal_data_collator,
    compute_metrics=comprehensive_metrics
)

# Remove '__index_level_0__' if it exists, as it's not expected by the model's forward method.
# This column is often added when converting a pandas DataFrame with a default index to a Hugging Face Dataset.
if '__index_level_0__' in tokenized_df_test.features:
    tokenized_df_test = tokenized_df_test.remove_columns(['__index_level_0__'])

# Perform evaluation on the prepared df_test dataset
eval_results = evaluator.evaluate(tokenized_df_test)

print("\nEvaluation Metrics for df_test (using model from Hugging Face):")
print(eval_results)


/tmp/ipython-input-4224098798.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  evaluator = Trainer(



Evaluation Metrics for df_test (using model from Hugging Face):
{'eval_loss': 4.171439170837402, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.49583333333333335, 'eval_f1': 0.4235235732009926, 'eval_precision': 0.49163763066202093, 'eval_recall': 0.49583333333333335, 'eval_auc': 0.4721527777777778, 'eval_runtime': 6.3171, 'eval_samples_per_second': 37.992, 'eval_steps_per_second': 2.374}


In [ ]:
# Move model to CPU for large test sets
model_from_hub.to("cuda")

# Predict
preds = evaluator.predict(
    tokenized_df_test,
    batch_size=8,           # smaller batch for memory safety
    use_cache=False,
    disable_tqdm=True
)

pred_labels = np.argmax(preds.predictions, axis=-1)
true_labels = preds.label_ids

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Plot
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for df_test")
plt.show()


RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: gpu

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Move model to CPU (optional if Gmodel_from_hub.to("gpu")PU is hanging)
model_from_hub.to("cpu")

# Make predictions
preds = evaluator.predict(tokenized_df_test, ignore_keys=None)  # disables hidden states etc.

# Convert logits to predicted labels
pred_labels = np.argmax(preds.predictions, axis=-1)
true_labels = preds.label_ids

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Plot
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for df_test")
plt.show()


In [ ]:
# --- Comprehensive evaluation metrics: accuracy, f1, precision, recall, auc ---
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

def metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    results = {}
    results["accuracy"] = float(accuracy_score(labels, preds))
    results["f1"] = float(f1_score(labels, preds, average="weighted"))
    results["precision"] = float(precision_score(labels, preds, average="weighted", zero_division=0))
    results["recall"] = float(recall_score(labels, preds, average="weighted", zero_division=0))
    # AUC (binary or macro-averaged for multiclass)
    try:
        if logits.shape[-1] == 2:
            # probability for positive class
            probs = np.exp(logits[:,1]) / np.exp(logits).sum(axis=1)
            results["auc"] = float(roc_auc_score(labels, probs))
        else:
            from sklearn.preprocessing import label_binarize
            lb = label_binarize(labels, classes=list(range(logits.shape[-1])))
            probs = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
            aucs = []
            for i in range(lb.shape[1]):
                try:
                    aucs.append(roc_auc_score(lb[:,i], probs[:,i]))
                except Exception:
                    pass
            results["auc"] = float(np.mean(aucs)) if aucs else None
    except Exception:
        results["auc"] = None
    return results

print("Comprehensive compute_metrics function added (accuracy, f1, precision, recall, auc).")

In [ ]:
# --- Ensure Trainer uses our metrics and that training runs on GPU if available ---
import torch
if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected; training will use CPU.")

# Attach the comprehensive metrics to the trainer (if trainer already exists). If not, reinstantiate trainer before training.
try:
    trainer.compute_metrics = metrics
    print("Attached compute_metrics to existing trainer.")
except NameError:
    print("Trainer not yet defined in notebook scope; ensure you pass compute_metrics=metrics when creating the Trainer instance.")

# Move model to device (Trainer also manages device automatically, this is explicit)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)